In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from nltk.corpus import stopwords
import gensim
import spacy

stop_words = stopwords.words('english')

In [ ]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format("../input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin",binary=True)
nlp = spacy.load("en_core_web_sm")

In [ ]:

def get_sentence_vector(sentence):
    doc = nlp(sentence)
    sent_vec = []
    for token in doc:
#         print(token)
        try:
            word_vec = word_vectors[str(token)]
    #         print(word_vec, type(word_vec))
            sent_vec.append(word_vec)
        except:
            continue
    sent_vec = np.asarray(sent_vec)
    return sent_vec.mean(axis=0)

In [ ]:
sentence = 'This is a test sentence.'
get_sentence_vector(sentence)

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
'''
This function defines the mean squared loss.
A built-in function from the sklearn package
'''
def mse_loss(y_true,y_pred):
    return sklearn.metrics.mean_squared_error(y_true,y_pred)

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.excerpt.values, train.target.values,
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
def get_sent_array(x):
    x_vec = []
    for sent in x:
        sent_vector = get_sentence_vector(sent)
        x_vec.append(sent_vector)
    return x_vec

In [ ]:
xtrain_vec = get_sent_array(xtrain)
xvalid_vec = get_sent_array(xvalid)

In [ ]:
clf = xgb.XGBRegressor()
clf.fit(np.asarray(xtrain_vec), ytrain)
predictions = clf.predict(np.asarray(xvalid_vec))

print ("logloss: %0.3f " % mse_loss(yvalid, predictions))

In [ ]:
xtest_vec = get_sent_array(list(test.excerpt))

In [ ]:
final_predictions = clf.predict(np.asarray(xtest_vec))

In [ ]:
final_predictions

In [ ]:
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
submission.target = list(final_predictions)
submission.to_csv("submission.csv", index=False)